In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 1.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7vth7faz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7vth7faz
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369408 sha256=fa189eb58de02a82e89d3ce61276e5c0c3874de88b0d0e3ee7efdae5ae2d743c
  Stored in directory: /tmp/pip-ephem-wheel-cache-rvfc_cq7/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip


In [ ]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
!pip install flax
import jax
import jax.numpy as jnp
import numpy as np
import torch
!pip install ml_collections
import ml_collections
import flax

torch:  1.13 ; cuda:  cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 KB 8.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ml_collections: filename=ml_collections-0.1.1-py3-none-any.whl size=945

In [ ]:
import clip
clip_model, preprocess = clip.load("ViT-B/16", device='cpu')

In [ ]:
torch_weights = {k: v for k, v in clip_model.visual.state_dict().items()}

In [ ]:
from tabulate import tabulate
table = []
for k in sorted(torch_weights):
  v = torch_weights[k]
  table.append((k, f'{v.shape}', f'{v.mean():.3f}', f'{v.std():.3f}'))
table_str = tabulate(
    table, tablefmt="pipe", headers=["Names", "shape", "mean", "std"])
print(table_str)

| Names                                         | shape                        |   mean |   std |
|:----------------------------------------------|:-----------------------------|-------:|------:|
| class_embedding                               | torch.Size([768])            |  0.003 | 0.234 |
| conv1.weight                                  | torch.Size([768, 3, 16, 16]) |  0     | 0.019 |
| ln_post.bias                                  | torch.Size([768])            |  0.192 | 0.31  |
| ln_post.weight                                | torch.Size([768])            |  0.951 | 0.102 |
| ln_pre.bias                                   | torch.Size([768])            | -0.001 | 0.073 |
| ln_pre.weight                                 | torch.Size([768])            |  0.393 | 0.455 |
| positional_embedding                          | torch.Size([197, 768])       | -0.007 | 0.028 |
| proj                                          | torch.Size([768, 512])       |  0     | 0.013 |
| transformer.resblo

In [ ]:
#@title Map layer names

def dfs(k, v, converted_torch_weight):
  """Recursively match weights."""
  if isinstance(v, jnp.ndarray):
    if k in converted_torch_weight:
      torch_data = converted_torch_weight[k]
      if len(v.shape) == 2 and 'embedding' not in k and 'rel_pos' not in k:
        torch_data = np.transpose(torch_data, (1, 0))
      if len(v.shape) == 4:
        if 'fpn_stride_16_8' in k:
          torch_data = np.transpose(torch_data, (2, 3, 0, 1))
        else:
          torch_data = np.transpose(torch_data, (2, 3, 1, 0))
      if torch_data.shape != v.shape:
        print('Wrong shape! {} {} {}'.format(
            k, torch_data.shape, v.shape))
    else:
      print(f'{k} not in checkpoint')
      torch_data = v
    return [(k, torch_data.shape)], torch_data
  lst, tree = [], {}
  for kk, vv in v.items():
    if isinstance(vv, jnp.ndarray) and (
        kk == 'kernel' or kk == 'scale' or kk == 'embedding'):
      if 'proposal_generator.scales' not in k:
        new_kk = 'weight'
      else:
        new_kk = kk
    else:
      new_kk = kk
    sub_lst, sub_tree = dfs(
        '{}.{}'.format(k, new_kk) if k else new_kk,
        vv,
        converted_torch_weight)
    lst.extend(sub_lst)
    tree[kk] = sub_tree
  return lst, tree

COMMEN_NAME_MAP = [
    ('transformer.', ''),
    ('resblocks', 'blocks'),
    ('in_proj_bias', 'qkv.bias'),
    ('in_proj_weight', 'qkv.weight'),
    ('out_proj', 'proj'),
    ('ln_1', 'norm1'),
    ('ln_2', 'norm2'),
    ('c_fc', 'fc1'),
    ('c_proj', 'fc2'),
    ('conv1', 'patch_embed.proj'),
    ('positional_embedding', 'pos_embed'),
]

def map_names(state_dict, name_map):
  """Change names according to a pre-defined map."""
  ret = {}
  for k, v in state_dict.items():
    new_k = k
    for ori_name, new_name in name_map:
      new_k = new_k.replace(ori_name, new_name)
    ret[new_k] = v
  return ret

converted_torch_weight = map_names(torch_weights, COMMEN_NAME_MAP)
remove_keys = ['class_embedding', 'ln_post.bias', 'ln_post.weight', 'proj']
for k in remove_keys:
  del converted_torch_weight[k]
converted_torch_weight['pos_embed'] = converted_torch_weight['pos_embed'][None]
converted_torch_weight = {k: v.numpy() for k, v in converted_torch_weight.items()}
# for k, v in converted_torch_weight.items():
#   print(k, v.shape)

In [ ]:
#@title JAX ViT implementation
"""ViTDet with simple FPN."""

import functools
from typing import Any, Optional

import flax.linen as nn
import jax
import jax.numpy as jnp
import ml_collections

KERNEL_INIT = {
    'normal': nn.initializers.normal(stddev=0.02),
}

__all__ = ['ViT', 'SimpleFeaturePyramid']


class Attention(nn.Module):
  """Multi-head Attention block with relative position embeddings.

  Attributes:
  dim (int): Number of input channels.
  num_heads (int): Number of attention heads.
  qkv_bias (bool:  If True, add a learnable bias to query, key, value.
  use_rel_pos (bool): If True, add relative positional embeddings to the
    attention map.
  rel_pos_zero_init (bool): If True, zero initialize relative positional
    parameters.
  input_size (int or None): Input resolution for calculating the relative
    positional parameter size.
  """
  dim: int
  num_heads: int = 8
  qkv_bias: bool = True
  use_rel_pos: bool = False
  rel_pos_zero_init: bool = True
  input_size: Optional[Any] = None
  kernel_init: str = 'normal'
  dtype: jnp.dtype = jnp.float32

  def get_rel_pos(self, q_size, k_size, rel_pos):
    """Get relative positional embeddings.

    Args:
      q_size (int): size of query q.
      k_size (int): size of key k.
      rel_pos (Tensor): relative position embeddings (L, C).
    Returns:
      Extracted positional embeddings according to relative positions.
    """
    max_rel_dist = int(2 * max(q_size, k_size) - 1)
    # Interpolate rel pos if needed.
    if rel_pos.shape[0] != max_rel_dist:
      # Interpolate rel pos.
      rel_pos_resized = jax.image.resize(
          rel_pos,
          shape=(max_rel_dist, rel_pos.shape[1]),
          method='linear',
      )
    else:
      rel_pos_resized = rel_pos

    # Scale the coords with short length if shapes for q and k are different.
    q_coords = jnp.arange(q_size)[:, None] * max(k_size / q_size, 1.0)
    k_coords = jnp.arange(k_size)[None, :] * max(q_size / k_size, 1.0)
    relative_coords = (q_coords - k_coords) + (k_size - 1) * max(
        q_size / k_size, 1.0)
    relative_coords = relative_coords.astype(jnp.int32).reshape(-1)
    return rel_pos_resized[relative_coords].reshape(q_size, k_size, -1)

  def add_decomposed_rel_pos(
      self, attn, q, rel_pos_h, rel_pos_w, q_size, k_size):
    """Calculate decomposed Relative Positional Embeddings from paper:`mvitv2`.

    Args:
      attn (Tensor): attention map.
      q (Tensor): query q in the attention layer with shape (B, q_h * q_w, C).
      rel_pos_h (Tensor): relative position embeddings (Lh, C) for height axis.
      rel_pos_w (Tensor): relative position embeddings (Lw, C) for width axis.
      q_size (Tuple): spatial sequence size of query q with (q_h, q_w).
      k_size (Tuple): spatial sequence size of key k with (k_h, k_w).
    Returns:
      attn (Tensor): attention map with added relative positional embeddings.
    """
    q_h, q_w = q_size
    k_h, k_w = k_size
    rh = self.get_rel_pos(q_h, k_h, rel_pos_h)
    rw = self.get_rel_pos(q_w, k_w, rel_pos_w)

    batch, _, dim = q.shape
    r_q = q.reshape(batch, q_h, q_w, dim)
    rel_h = jnp.einsum('bhwc,hkc->bhwk', r_q, rh)
    rel_w = jnp.einsum('bhwc,wkc->bhwk', r_q, rw)

    attn = (
        attn.reshape(batch, q_h, q_w, k_h, k_w) + rel_h[
            :, :, :, :, None] + rel_w[:, :, :, None, :]
    ).reshape(batch, q_h * q_w, k_h * k_w)

    return attn

  @nn.compact
  def __call__(self, x):
    batch, height, width, _ = x.shape
    head_dim = self.dim // self.num_heads
    qkv = nn.Dense(self.dim * 3, use_bias=self.qkv_bias, name='qkv')(
        x)  # batch x height x width x 3dim
    qkv = qkv.reshape(batch, height * width, 3, self.num_heads, -1).transpose(
        2, 0, 3, 1, 4)  # 3 x batch x num_heads x num_tokens x D
    qkv = qkv.reshape(3, batch * self.num_heads, height * width, -1)
    q, k, v = qkv[0], qkv[1], qkv[2]  # [batch * num_heads, num_tokens, D]
    attn = (q * (head_dim ** -0.5)) @ k.transpose(
        0, 2, 1)  # [batch * num_heads, num_tokens, num_tokens]
    if self.use_rel_pos:
      rel_pos_h = self.param(
          'rel_pos_h', nn.initializers.zeros,
          (2 * self.input_size[0] - 1, head_dim))
      rel_pos_w = self.param(
          'rel_pos_w', nn.initializers.zeros,
          (2 * self.input_size[0] - 1, head_dim))
      attn = self.add_decomposed_rel_pos(
          attn, q, rel_pos_h, rel_pos_w,
          (height, width), (height, width))
    attn = jax.nn.softmax(attn)
    x = (attn @ v).reshape(batch, self.num_heads, height, width, -1).transpose(
        0, 2, 3, 1, 4).reshape(batch, height, width, -1)
    x = nn.Dense(self.dim, name='proj')(x)
    return x

def quick_gelu(x: jnp.ndarray) -> jnp.ndarray:
  return x * jax.nn.sigmoid(1.702 * x)


class Mlp(nn.Module):
  """Multilayer perceptron."""

  hidden_features: int
  out_features: int
  kernel_init: str = 'normal'
  dtype: jnp.dtype = jnp.float32

  @nn.compact
  def __call__(self, x):
    x = nn.Dense(
        self.hidden_features, dtype=self.dtype,
        kernel_init=KERNEL_INIT[self.kernel_init], name='fc1')(x)
    # x = nn.gelu(x, approximate=False)
    x = quick_gelu(x)
    x = nn.Dense(
        self.out_features, dtype=self.dtype,
        kernel_init=KERNEL_INIT[self.kernel_init], name='fc2')(x)
    return x


class Block(nn.Module):
  """Transformer blocks with support of window attention and residual blocks.

  Attributes:
    dim (int): Number of input channels.
    num_heads (int): Number of attention heads in each ViT block.
    mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
    qkv_bias (bool): If True, add a learnable bias to query, key, value.
    drop_path (float): Stochastic depth rate.
    use_rel_pos (bool): If True, add relative positional embeddings to the
      attention map.
    rel_pos_zero_init (bool): If True, zero initialize relative positional
      parameters.
    window_size (int): Window size for window attention blocks. If it equals 0,
      then not use window attention.
    input_size (int or None): Input resolution for calculating the relative
      positional parameter size.
  """
  dim: int
  num_heads: int
  mlp_ratio: float = 4.0
  qkv_bias: bool = True
  drop_path: float = 0.0
  use_rel_pos: bool = False
  rel_pos_zero_init: bool = True
  window_size: int = 0
  input_size: Optional[int] = None
  layer_scale_init_value: float = -1.0
  kernel_init: str = 'normal'
  dtype: jnp.dtype = jnp.float32

  def window_partition(self, x):
    """Partition into non-overlapping windows with padding if needed.

    Args:
      x (array): input tokens with [B, H, W, C].
    Returns:
      windows: windows after partition with [B * num_windows, window_size,
        window_size, C].
      (Hp, Wp): padded height and width before partition
    """
    batch, h, w, c = x.shape

    pad_h = (self.window_size - h % self.window_size) % self.window_size
    pad_w = (self.window_size - w % self.window_size) % self.window_size
    if pad_h > 0 or pad_w > 0:
      x = jnp.pad(
          x, ((0, 0), (0, pad_w), (0, pad_h), (0, 0)),
          'constant', constant_values=0)
    hp, wp = h + pad_h, w + pad_w

    x = x.reshape(
        batch, hp // self.window_size, self.window_size,
        wp // self.window_size, self.window_size, c)
    windows = x.transpose(0, 1, 3, 2, 4, 5).reshape(
        -1, self.window_size, self.window_size, c)
    return windows, (hp, wp)

  def window_unpartition(self, windows, pad_hw, hw):
    """Window unpartition into original sequences and removing padding.

    Args:
      windows (array): inputs: [B * num_windows, window_size, window_size, C].
      pad_hw (Tuple): padded height and width (Hp, Wp).
      hw (Tuple): original height and width (H, W) before padding.

    Returns:
      x: unpartitioned sequences with [B, H, W, C].
    """
    hp, wp = pad_hw
    h, w = hw
    batch = windows.shape[0] // (
        hp * wp // self.window_size // self.window_size)
    x = windows.reshape(
        batch,
        hp // self.window_size, wp // self.window_size,
        self.window_size, self.window_size, -1)
    x = x.transpose(0, 1, 3, 2, 4, 5).reshape(batch, hp, wp, -1)
    if hp > h or wp > w:
      x = x[:, :h, :w, :]
    return x

  def get_keep_pattern(self,
                       x: jnp.ndarray,
                       deterministic: bool) -> jnp.ndarray:
    """DropPath Layer."""
    if not deterministic and self.drop_path:
      shape = (x.shape[0],) + (1,) * (x.ndim - 1)
      drop_pattern = jax.random.bernoulli(
          self.make_rng('dropout'), self.drop_path, shape).astype(self.dtype)
      keep_pattern = (1. - drop_pattern)
      if self.drop_path < 1.:
        keep_pattern = keep_pattern / (1. - self.drop_path)
      return keep_pattern
    else:
      return 1.0

  @nn.compact
  def __call__(self, x, train = False):
    shortcut = x
    ln = functools.partial(nn.LayerNorm, epsilon=1e-6)
    x = ln(name='norm1')(x)
    # Window partition
    if self.window_size > 0:
      h, w = x.shape[1], x.shape[2]
      x, pad_hw = self.window_partition(x)

    x = Attention(
        self.dim,
        num_heads=self.num_heads,
        qkv_bias=self.qkv_bias,
        use_rel_pos=self.use_rel_pos,
        rel_pos_zero_init=self.rel_pos_zero_init,
        input_size=self.input_size if self.window_size == 0 else (
            self.window_size, self.window_size),
        name='attn')(x)
    # Reverse window partition
    if self.window_size > 0:
      x = self.window_unpartition(x, pad_hw, (h, w))

    if self.layer_scale_init_value > 0:
      gamma_1 = self.param(
          'gamma_1',
          nn.initializers.constant(self.layer_scale_init_value),
          (self.dim))
      x = x * gamma_1[..., :]
    x = shortcut + self.get_keep_pattern(x, not train) * x

    y = ln(name='norm2')(x)
    y = Mlp(
        int(self.dim * self.mlp_ratio),
        self.dim,
        kernel_init=self.kernel_init,
        dtype=self.dtype,
        name='mlp')(y)
    if self.layer_scale_init_value > 0:
      gamma_2 = self.param(
          'gamma_2',
          nn.initializers.constant(self.layer_scale_init_value),
          (self.dim))
      y = y * gamma_2[..., :]
    x = x + self.get_keep_pattern(y, not train) * y
    return x


class ViT(nn.Module):
  """This module implements Vision Transformer (ViT) backbone in paper:`vitdet`.

  "Exploring Plain Vision Transformer Backbones for Object Detection",
  https://arxiv.org/abs/2203.16527

  Attributes:
    img_size (int): Input image size.
    patch_size (int): Patch size.
    in_chans (int): Number of input image channels.
    embed_dim (int): Patch embedding dimension.
    depth (int): Depth of ViT.
    num_heads (int): Number of attention heads in each ViT block.
    mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
    qkv_bias (bool): If True, add a learnable bias to query, key, value.
    drop_path_rate (float): Stochastic depth rate.
    use_abs_pos (bool): If True, use absolute positional embeddings.
    use_rel_pos (bool): If True, add relative positional embeddings to the
      attention map.
    rel_pos_zero_init (bool): If True, zero initialize relative positional
      parameters.
    window_size (int): Window size for window attention blocks.
    window_block_indexes (list): Indexes for blocks using window attention.
    pretrain_img_size (int): input image size for pretraining models.
    pretrain_use_cls_token (bool): If True, pretrainig models use class token.
  """
  img_size: int = 1024
  patch_size: int = 16
  in_chans: int = 3
  embed_dim: int = 768
  depth: int = 12
  num_heads: int = 12
  mlp_ratio: float = 4.0
  qkv_bias: bool = True
  drop_path_rate: float = 0.1
  use_abs_pos: bool = True
  use_rel_pos: bool = True
  rel_pos_zero_init: bool = True
  window_size: int = 14
  window_block_indexes: Any = (0, 1, 3, 4, 6, 7, 9, 10)
  pretrain_img_size: int = 224
  pretrain_use_cls_token: bool = True
  layer_scale_init_value: float = -1.0
  kernel_init: str = 'normal'
  with_cls_token: bool = False
  use_ln_pre: bool = False
  dtype: jnp.dtype = jnp.float32

  def _get_abs_pos(self, abs_pos, hw):
    """Calculate absolute positional embeddings.

    If needed, resize embeddings and remove cls_token dimension for the original
      embeddings.
    Args:
      abs_pos (array): absolute positional embeddings with (1, num_position, C).
      hw (Tuple): size of input image tokens.
    Returns:
      Absolute positional embeddings after processing with shape (1, H, W, C)
    """
    h, w = hw
    if self.pretrain_use_cls_token:
      abs_pos = abs_pos[:, 1:]
    xy_num = abs_pos.shape[1]
    size = int(xy_num ** 0.5)
    assert size * size == xy_num
    abs_pos = abs_pos.reshape(abs_pos.shape[0], size, size, -1)
    if size != h or size != w:
      new_abs_pos = jax.image.resize(
          abs_pos,
          (abs_pos.shape[0], h, w, abs_pos.shape[3]),
          method='bicubic',
      )
    else:
      new_abs_pos = abs_pos
    return new_abs_pos

  @nn.compact
  def __call__(self, x: jnp.ndarray, train: bool = False):
    print('input', x.shape)
    x = nn.Conv(
        self.embed_dim, (self.patch_size, self.patch_size),
        strides=(self.patch_size, self.patch_size),
        padding='VALID',
        name='patch_embed.proj')(x)
    print('after conv', x.shape, x[0, 0, 0, :10])
    if self.use_abs_pos:
      num_patches = (self.pretrain_img_size // self.patch_size) ** 2
      num_positions = (
          num_patches + 1) if self.pretrain_use_cls_token else num_patches
      pos_embed = self.param(
          'pos_embed', nn.initializers.zeros,
          (1, num_positions, self.embed_dim))
      x = x + self._get_abs_pos(pos_embed, (x.shape[1], x.shape[2]))
    print('after pos emb', x.shape, x[0, 0, 0, :10])
    if self.with_cls_token:
      cls_token = self.param(
          'cls_token', nn.initializers.zeros, (1, 1, self.embed_dim))
    dp_rates = [
        self.drop_path_rate * i / (self.depth - 1) for i in range(self.depth)]
    if self.use_ln_pre:
      x = nn.LayerNorm(name='ln_pre')(x)
    print('after ln pre', x.shape, x[0, 0, 0, :10])
    for i in range(self.depth):
      x = Block(
          dim=self.embed_dim,
          num_heads=self.num_heads,
          mlp_ratio=self.mlp_ratio,
          qkv_bias=self.qkv_bias,
          drop_path=dp_rates[i],
          use_rel_pos=self.use_rel_pos,
          rel_pos_zero_init=self.rel_pos_zero_init,
          window_size=self.window_size if i in self.window_block_indexes else 0,
          input_size=(
              self.img_size // self.patch_size,
              self.img_size // self.patch_size),
          layer_scale_init_value=self.layer_scale_init_value,
          name=f'blocks.{i}',
          )(x, train=train)
      print(f'after block {i}', x.shape, x[0, 0, 0, :10])
    return x


SIZE_CONFIGS = {
    'B': (768, 12, 12, 0.1, (0, 1, 3, 4, 6, 7, 9, 10)),
    'L': (1024, 24, 16, 0.4, (
        0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22)),
    'H': (1280, 32, 16, 0.5, (
        0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21,
        22, 24, 25, 26, 27, 28, 29, 30)),
}


In [ ]:
#@title
import ml_collections
backbone_args = ml_collections.ConfigDict()
sz = 'B'  # backbone_args.pop('size', 'B')
dim, depth, num_heads, dp, window_block_indexes = SIZE_CONFIGS[sz]
backbone_args['embed_dim'] = backbone_args.get(
    'embed_dim', dim)
backbone_args['depth'] = backbone_args.get('depth', depth)
backbone_args['num_heads'] = backbone_args.get(
    'num_heads', num_heads)
backbone_args['drop_path_rate'] = backbone_args.get(
    'drop_path_rate', dp)
backbone_args['window_block_indexes'] = backbone_args.get(
    'window_block_indexes', window_block_indexes)
backbone_args['use_rel_pos'] = True
backbone_args['use_ln_pre'] = True
vit_model = ViT(**backbone_args)

rng = {'dropout': jax.random.PRNGKey(0), 'params': jax.random.PRNGKey(0)}
input = jax.random.normal(jax.random.PRNGKey(0), (1, 1024, 1024, 3))
vit_vars = vit_model.init(rng, input)

input (1, 1024, 1024, 3)
after conv (1, 64, 64, 768) [ 0.47106028  1.3287854   0.83192176 -0.83944285  1.0247202  -0.48860377
 -0.12125206  0.8328862   1.0072198  -0.8418225 ]
after pos emb (1, 64, 64, 768) [ 0.47106028  1.3287854   0.83192176 -0.83944285  1.0247202  -0.48860377
 -0.12125206  0.8328862   1.0072198  -0.8418225 ]
after ln pre (1, 64, 64, 768) [ 0.43739814  1.274982    0.7897858  -0.84233147  0.97805685 -0.49973089
 -0.1410054   0.7907276   0.9609675  -0.8446553 ]
after block 0 (1, 64, 64, 768) [ 0.9527342   1.6842506   1.1183987  -0.620758    1.0156078  -0.43444997
  0.40971595  0.77097815  1.5544728  -1.0375676 ]
after block 1 (1, 64, 64, 768) [ 0.9354738   1.1514326   1.3138807  -0.7885489   0.6950543  -0.48360264
 -0.47691417  1.2413012   1.6396505  -0.9739183 ]
after block 2 (1, 64, 64, 768) [ 0.54650915  0.7949942   1.1095765  -0.34691972  0.42791653 -0.5394424
 -0.71003187  1.6934314   1.8298969  -0.83660793]
after block 3 (1, 64, 64, 768) [ 0.40429586  0.07155712 

In [ ]:
ret, tree = dfs('', vit_vars['params'], converted_torch_weight)
num_params = 0
for k, v in converted_torch_weight.items():
  num_params += np.prod(v.shape)
print('#params in loaded model:', num_params)
num_params = 0
for k, v in ret:
  if 'rel_pos' not in k:
    num_params += np.prod(v)
print('#params in converted model:', num_params)
print('#params in converted model w.o proj bias:', num_params - 768)

patch_embed.proj.bias not in checkpoint
blocks.0.attn.rel_pos_h not in checkpoint
blocks.0.attn.rel_pos_w not in checkpoint
blocks.1.attn.rel_pos_h not in checkpoint
blocks.1.attn.rel_pos_w not in checkpoint
blocks.2.attn.rel_pos_h not in checkpoint
blocks.2.attn.rel_pos_w not in checkpoint
blocks.3.attn.rel_pos_h not in checkpoint
blocks.3.attn.rel_pos_w not in checkpoint
blocks.4.attn.rel_pos_h not in checkpoint
blocks.4.attn.rel_pos_w not in checkpoint
blocks.5.attn.rel_pos_h not in checkpoint
blocks.5.attn.rel_pos_w not in checkpoint
blocks.6.attn.rel_pos_h not in checkpoint
blocks.6.attn.rel_pos_w not in checkpoint
blocks.7.attn.rel_pos_h not in checkpoint
blocks.7.attn.rel_pos_w not in checkpoint
blocks.8.attn.rel_pos_h not in checkpoint
blocks.8.attn.rel_pos_w not in checkpoint
blocks.9.attn.rel_pos_h not in checkpoint
blocks.9.attn.rel_pos_w not in checkpoint
blocks.10.attn.rel_pos_h not in checkpoint
blocks.10.attn.rel_pos_w not in checkpoint
blocks.11.attn.rel_pos_h not in ch

In [ ]:
flattened_tree = flax.traverse_util.flatten_dict(tree)
table = []
for k in sorted(flattened_tree):
  if 'rel_pos_h' in k or 'rel_pos_w' in k:
    continue
  v = flattened_tree[k]
  table.append((k, f'{v.shape}', f'{v.mean():.3f}', f'{v.std():.3f}'))
table_str = tabulate(
    table, tablefmt="pipe", headers=["Names", "shape", "mean", "std"])
print(table_str)

| Names                                   | shape                        |   mean |   std |
|:----------------------------------------|:-----------------------------|-------:|------:|
| ('blocks.0', 'attn', 'proj', 'bias')    | torch.Size([768])            |  0.001 | 0.043 |
| ('blocks.0', 'attn', 'proj', 'kernel')  | torch.Size([768, 768])       | -0     | 0.011 |
| ('blocks.0', 'attn', 'qkv', 'bias')     | torch.Size([2304])           |  0.008 | 0.538 |
| ('blocks.0', 'attn', 'qkv', 'kernel')   | torch.Size([768, 2304])      | -0     | 0.014 |
| ('blocks.0', 'mlp', 'fc1', 'bias')      | torch.Size([3072])           | -0.455 | 0.336 |
| ('blocks.0', 'mlp', 'fc1', 'kernel')    | torch.Size([768, 3072])      | -0     | 0.015 |
| ('blocks.0', 'mlp', 'fc2', 'bias')      | torch.Size([768])            | -0.004 | 0.074 |
| ('blocks.0', 'mlp', 'fc2', 'kernel')    | torch.Size([3072, 768])      |  0     | 0.009 |
| ('blocks.0', 'norm1', 'bias')           | torch.Size([768])            | -0.01

In [ ]:
from flax.training import checkpoints
out_path = 'clip_b_16'
checkpoints.save_checkpoint(out_path, {'params': tree}, 0)
# from google.colab import files
# files.download(f'{out_path}/checkpoint_0')

'clip_b_16/checkpoint_0'